## Raw to csv preprocessing

In [15]:
from utility.preprocessing2 import clean_data
DATA_SIZE=10000
BASE_PATH=f'data/{DATA_SIZE}'
clean_data(BASE_PATH)

### Feature engineering 

In [1]:
from utility.preprocessing2 import clean_data,lower_case, preprocessing, upsample
from sklearn.model_selection import train_test_split
import pandas as pd
import os
DATA_SIZE=100000
df=pd.read_csv(f"data/process/final_{DATA_SIZE}.csv")

In [2]:
# check data len 
DAYS=30
print(f"total Admission : {len(df)}")
print(f"admit within {DAYS} : {len(df[df['days']<=DAYS])} ")


total Admission : 261760
admit within 30 : 1332 


In [3]:
all_feature=['PatientGender', 'PatientRace','Age',
'PatientMaritalStatus', 'PatientLanguage',
'PatientPopulationPercentageBelowPoverty', 'AdmissionID',
'PrimaryDiagnosisCode', 'PrimaryDiagnosisDescription', 'Target', ]

### process_df and train_df we can use for evaluation

In [9]:
#convert df in lower case
df=lower_case(df)
process_df=preprocessing(df,DAYS,all_feature) # whole dataset you will for evaluation
train_df=upsample(process_df,10000) #upsample low class

In [21]:
train_df['Target'].value_counts()

1    10000
0    10000
Name: Target, dtype: int64

### Train model and save

In [ ]:
from model.model2 import train_model
model,test,train = train_model(train_df)

Fitting 30 folds for each of 100 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  6.1min
/home/server/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 15.7min


,PatientGender,PatientRace,Age,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,Target,y_pred
170675,female,white,29,married,english,17.05,2,c49.4,malignant neoplasm of connective and soft tiss...,0,0
105583,female,african american,21,unknown,icelandic,6.73,2,m05.552,rheumatoid polyneuropathy with rheumatoid arth...,0,0
76765,male,unknown,65,married,english,17.16,3,j84.842,pulmonary interstitial glycogenosis,0,0
115454,male,white,68,single,english,10.65,4,m90.671,"osteitis deformans in neoplastic diseases, rig...",0,0
119471,female,white,32,separated,english,12.23,2,m02.351,"reiter's disease, right hip",1,1
...,...,...,...,...,...,...,...,...,...,...,...
220002,female,african american,25,single,english,11.02,2,e10.621,type 1 diabetes mellitus with foot ulcer,0,0
61660,female,white,20,married,english,15.05,1,g91.4,hydrocephalus in diseases classified elsewhere,0,0
227665,male,white,40,single,english,18.89,4,f13,"sedative, hypnotic, or anxiolytic related diso...",0,0
71334,female,white,32,married,english,12.29,3,m90.521,osteonecrosis in diseases classified elsewhere...,1,1


### Evaluation 

In [11]:
from testing.evaluation2 import evaluation_data
# use train_df or process_df  set True to get csv for manual comparison 
df_test=evaluation_data(train_df.copy(),'save_model/model.pkl',True)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator OneHotEncoder from 

Precision: 0.9919632606199771,Recall: 0.9504


In [86]:
# df_test=evaluation_data(train_df,'save_model/model.pkl')
print(f"total={len(df_test)}\n")
print(df_test.Target.value_counts())
print(f"\n0 <-> 0 = {len(df_test[(df_test['Target']==0) & (df_test['y_pred']==0)])}")
print(f"1 <-> 1 = {len(df_test[(df_test['Target']==1) & (df_test['y_pred']==1)])}")
print(f"0 <-> 1 = {len(df_test[(df_test['Target']==0) & (df_test['y_pred']==1)])}")
print(f"1 <-> 0 = {len(df_test[(df_test['Target']==1) & (df_test['y_pred']==0)])}")

total=261760

0    260428
1      1332
Name: Target, dtype: int64

0 <-> 0 = 241943
1 <-> 1 = 1276
0 <-> 1 = 18485
1 <-> 0 = 56


### inferences 

In [35]:
import pandas as pd
data={'PatientGender':['male','female'],
      'PatientRace':['white','white'],
      'Age':[41,32],
      'PatientMaritalStatus':['single','single'],
      'PatientLanguage':['english','english'],
      'PatientPopulationPercentageBelowPoverty':[11.53,12.3],
      'AdmissionID':[2,1],
      'PrimaryDiagnosisCode':['t70.3','t70.3'],
      'PrimaryDiagnosisDescription':['Caisson disease [decompression sickness]','Caisson']
     }
test_df=pd.DataFrame(data)

,PatientGender,PatientRace,Age,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
11965,male,african american,60,married,spanish,15.97,2,f30.12,"manic episode without psychotic symptoms, mode..."
157685,male,asian,27,married,english,12.85,2,j17,pneumonia in diseases classified elsewhere
106572,female,african american,46,married,icelandic,19.53,4,m51.0,"thoracic, thoracolumbar and lumbosacral interv..."
181226,female,african american,28,married,spanish,86.07,3,c94.31,"mast cell leukemia, in remission"
252936,male,asian,24,married,icelandic,10.13,1,m06.26,"rheumatoid bursitis, knee"
...,...,...,...,...,...,...,...,...,...
21791,female,white,74,unknown,unknown,15.18,4,m06.231,"rheumatoid bursitis, right wrist"
230511,female,asian,69,married,unknown,11.46,6,o9a.1,"malignant neoplasm complicating pregnancy, chi..."
148081,male,white,76,single,english,1.65,4,k57,diverticular disease of intestine
108154,male,african american,26,single,english,12.47,1,m90.84,"osteopathy in diseases classified elsewhere, hand"


In [37]:
from testing.evaluation2 import infer
infer(pd.DataFrame(test_df))

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator OneHotEncoder from 

['No Need To Admit', 'No Need To Admit']